<div style="background-image: url('https://i.pinimg.com/1200x/45/3a/06/453a06bdc2b2d27d8329857061537124.jpg'); 
            background-size: cover; 
            background-position: center; 
            padding: 30px; 
            text-align: center; 
            border-radius: 8px;">
    <h1 style="color: white; 
               font-size: 28px; 
               font-weight: bold; 
               text-shadow: 2px 2px 4px rgba(0,0,0,0.8), 
                            -1px -1px 2px rgba(0,0,0,0.8);
               margin: 0;
               font-family: 'Arial', sans-serif;">
        PROYECTO: MODELO DE RIESGO
    </h1>
</div>

- María José Castillo Silva
- Juan David Bocanegra Vargas

In [6]:
# Permite ajustar la anchura de la parte útil de la libreta (reduce los márgenes) y omitir warnings
import warnings
warnings.filterwarnings("ignore")

from IPython.display import HTML
display(HTML("<style>.container{ width:98% }</style>"))

# 1. Problema a Analizar

# ¿Cuál es la probabilidad de riesgo de default asociada a cada cliente, dadas sus diferentes características?.

La principal fuente de datos es Datacredito Experian, quienes envían información de los clientes actuales de la Entidad, incluyendo las siguientes variables: “Acierta Advance”, score de crédito del sector financiero, saldos, estados de productos crediticios y también información demografica como edad, sexo, entre otras.

# 2. Impacto del Problema

Actualmente en las áreas de riesgo de crédito en el sector bancario, se definen constantemente políticas que permiten soportar la toma de decisiones en la originación  de productos, que en la medida de lo posible, estén enmarcadas en la agilidad y precisión de la respuesta a clientes, y vayan en línea con el apetito financiero propuesto por la Junta Directiva.

# 3. Datos, primer análisis exploratorio

## Instalación de Librerias

In [ ]:
#!pip install eli5
#!pip install pandas
#!pip install numpy
#!pip install seaborn
#!pip install yellowbrick
#!pip install xgboost
#!pip install shap
#!pip install matplotlib
#!pip install scikit-learn
#!pip install imbalanced-learn

## Cargar Librerías

In [ ]:
%pip install -U scikit-learn==1.5.2 imbalanced-learn==0.12.3 yellowbrick==1.5

In [ ]:
# ===============================
# Núcleo científico / utilidades
# ===============================
import warnings; warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ===============================
# Modelado y validación (scikit-learn)
# ===============================
from sklearn.model_selection import (
    train_test_split, StratifiedKFold, RandomizedSearchCV, cross_val_score
)
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, average_precision_score, roc_curve, precision_recall_curve,
    confusion_matrix
)
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier

# ===============================
# Desbalanceo de clases (imbalanced-learn)
# ===============================
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

# ===============================
# Gradient Boosting (XGBoost)
# ===============================
from xgboost import XGBClassifier

# ===============================
# Estadística (SciPy y Statsmodels)
# ===============================
from scipy import stats
from scipy.stats import shapiro, normaltest, ttest_ind, mannwhitneyu
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor  # VIF

# ===============================
# Otros apoyos
# ===============================
from collections import Counter

# ===============================
# Opcionales de interpretación/visual
# ===============================
import shap
from yellowbrick.classifier import ROCAUC, ConfusionMatrix, ClassificationReport
import eli5

# ===============================
# Estilo de gráficos
# ===============================
plt.style.use("ggplot")


## Importar Datos

In [ ]:
datos = pd.read_csv('base_modelo_40k.csv', sep=',')
datos.info()

In [ ]:
datos

In [ ]:
print(datos.shape)

* Inicialmente se contemplaron 40 variables en estudio, mixtas entre categóricas y numéricas

## Limpieza y Armonización

In [ ]:
### CONVERSIÓN TIPOS DE FORMATO
datos['CRED_REESTRUCTURADO']=datos['CRED_REESTRUCTURADO'].astype('object')
datos['TIENE_HIPOTECA']=datos['TIENE_HIPOTECA'].astype('object')

In [ ]:
### Conteo valores nulos
datos.isnull().sum()

* Por criterio experto, se consideran no necesarias las variables asociadas a la identificación del cliente, como el tipo de Id, el número de identificación y la fecha de evaluación; también se elimina la variable "Acierta_plus" ya que existe la variable "Advance".

In [ ]:
#ELIMINAR COLUMNAS NO NECESARIAS
datos=datos.drop(labels='ID',axis=1)
datos

In [ ]:
### Dimensión base de datos
print(datos.shape)

## Análisis Exploratorio

* La Variable de interés es la variable llamada VAR_DEP, que es 1 si el cliente tuvo una mora mayor a 90 días en los doce meses siguientes al desembolso del credito y 0 si ha estado al día.

In [ ]:
### Proporción de clientes en Mora
datos.groupby('CLIENTE_MORA').size()/datos['CLIENTE_MORA'].count()*100

* La base con 54.759 clientes, contiene un 7.08% de clientes en mora y un 92.92% de clientes al día

In [ ]:
#Separación de Bases
datos_0 = datos[datos['CLIENTE_MORA'] == 0]
datos_1 = datos[datos['CLIENTE_MORA'] == 1]

#Función de densidad 
datos_0.SCORE_DATACREDITO.plot.density(color='green',label='Clientes Al día') 
datos_1.SCORE_DATACREDITO.plot.density(color='red',label='Clientes en Mora')
plt.legend()
plt.xlabel("Score Datacredito")
plt.ylabel('Probabilidad numérica')
plt.title('Score Datacredito Clientes al día y en mora')

* Se evidencia que los clientes que han tenido una mora de 90 días o más en los 12 ultimos meses tienen un Score Advance (cálculo por datacredito) menor a los clientes que han estado al día. También se resalta que, la mayoría de clientes en mora tienen un Score Advance entre 500 a 850, en cambio los clientes que han estado al día tienen un score adnvance entre 650 a 950.

### Pruebas estadísticas

In [ ]:
# Test de normalidad Shapiro-Wilk
print("Prueba Shapiro-Wilk Población al día:",shapiro(datos_0['SCORE_DATACREDITO']))
print("Prueba Shapiro-Wilk Población en mora:",shapiro(datos_1['SCORE_DATACREDITO']))

In [ ]:
# Diferencia de medias
def dif_medias (df1, df2, alfa):
    stat, p = ttest_ind(df1,df2, equal_var = False)
    print("Statistic=%.3f, p=%.3f" % (stat,p))

dif_medias (datos_0['SCORE_DATACREDITO'], datos_1['SCORE_DATACREDITO'], 0.05)

In [ ]:
# Prueba no parametrcia Mann-Whitney para comparar la variable ADVANCE en los dos grupos
stat, p = mannwhitneyu(datos_0['SCORE_DATACREDITO'], datos_1['SCORE_DATACREDITO'], alternative='two-sided')

print("Mann-Whitney U Test")
print(f"Estadístico U = {stat:.3f}, p-valor = {p:.3e}")

# Interpretación rápida
alpha = 0.05
if p < alpha:
    print("👉 Se rechaza H0: Las distribuciones de ADVANCE en los dos grupos son diferentes.")
else:
    print("👉 No se rechaza H0: No hay evidencia de diferencia significativa entre los grupos.")

* Existe evidencia estadísticamente significativa para afirmar que las medias de la variable ADVANCE en los dos grupos comparados (datos_0 y datos_1) son diferentes.

In [ ]:
datos_0.ANT_SF.plot.density(color='green',label='Clientes Al día') 
datos_1.ANT_SF.plot.density(color='red',label='Clientes en Mora')
plt.legend()
plt.xlabel("Antigüedad Financiera")
plt.ylabel('Probabilidad numérica')
plt.title('Antigüedad Financiera de Clientes al día y en mora')

* Se evidencia como lo clientes que estan al día tienden  a tener mayor antiguedad en el sector financiero a comparación de los cliente en mora.

In [ ]:
# prueba K-S
stats.ks_2samp(datos_0['ANT_SF'], datos_1['ANT_SF'])

👉 Los clientes al día tienden a tener una mayor antigüedad en el sistema financiero que los clientes en mora.

In [ ]:
datos_0.EDAD.plot.density(color='green',label='Clientes Al día') 
datos_1.EDAD.plot.density(color='red',label='Clientes en Mora')
plt.legend()
plt.xlabel("Edad")
plt.ylabel('Probabilidad numérica')
plt.title('Edad Clientes al día y en mora')

* Los clientes en mora tienden a concentrarse en edades entre los 30 a los 40 años, en cambio los clientes al día tienden a ser de mayor edad.

In [ ]:
pd.crosstab(datos['SEXO'], datos['CLIENTE_MORA'], normalize='index')*100

* Se evidencia que las mujeres pagan mejor que los hombres, dado que el porcentaje de incumplimiento de las mujeres es 6.7% en cambio de los hombres es de 7.5%.

In [ ]:
datos_0.INGRESO.plot.density(color='green',label='Clientes Al día') 
datos_1.INGRESO.plot.density(color='red',label='Clientes en Mora')
plt.legend()
plt.xlabel("Ingresos")
plt.ylabel('Probabilidad numérica')
plt.title('Ingresos de Clientes al día y en mora')

* Se evidencia que no existen diferencias de ingreso respecto al incumplimiento de pago de los clientes.

In [ ]:
datos_0.INGRESO.plot.density(color='green',label='Clientes Al día') 
datos_1.INGRESO.plot.density(color='red',label='Clientes en Mora')
plt.legend()
plt.xlabel("Ingresos")
plt.ylabel('Probabilidad numérica')
plt.title('Ingresos de Clientes al día y en mora')

### Correlación

In [ ]:
# Matriz de correlación compacta con encabezados verticales sin superposición
corr = datos.select_dtypes(include='number').corr().round(2)

display(
    corr.style
        .format("{:.2f}")
        .background_gradient(cmap="BrBG")
        # fuente y padding pequeños en todo
        .set_properties(**{"font-size":"8pt", "padding":"2px"})
        # estilos específicos de encabezados
        .set_table_styles([
            # columnas: vertical, ancho fijo pequeño, sin quiebre ni solape
            {"selector":"th.col_heading",
             "props":[
                 ("writing-mode","vertical-rl"),
                 ("text-orientation","mixed"),
                 ("width","24px"), ("min-width","24px"), ("max-width","24px"),
                 ("height","140px"),
                 ("white-space","nowrap"),
                 ("overflow","hidden"),
                 ("text-overflow","clip"),
                 ("font-size","7pt"),
                 ("padding","6px 2px")
             ]},
            # filas: fuente pequeña
            {"selector":"th.row_heading",
             "props":[("font-size","7pt"), ("white-space","nowrap")]}
        ])
)

* En la matriz de correlación se evidencia que las variables que más estan relacionadas con la variable de incumplimiento de pago es el score Advance generado por Datacredito y en segunda medida variables como los saldos de los creditos, los estados de mora y la antiguedad en el sector financiero. 


### Eliminación de variables VIF

In [ ]:
# 1) Solo numéricas y limpieza mínima
num = datos.select_dtypes(include='number').copy()
num = num.dropna(axis=1, how='all')                 # quita columnas totalmente vacías
num = num.loc[:, num.nunique()>1]                   # quita columnas con varianza 0
num = num.fillna(num.median(numeric_only=True))     # imputación simple (si hay NA)

# 2) Matriz para VIF (no necesita estandarizar)
X = sm.add_constant(num, has_constant='add')

# 3) Calcular VIF (omitimos la constante en el resultado)
vif_vals = [variance_inflation_factor(X.values, i) for i in range(1, X.shape[1])]
vifs = (pd.DataFrame({'Variable': num.columns, 'VIF': vif_vals})
          .replace([np.inf, -np.inf], np.nan)
          .sort_values('VIF', ascending=False)
          .reset_index(drop=True))

# 4) Mostrar
display(vifs.style.format({'VIF':'{:.2f}'}))


# Pipelines

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, RobustScaler
import numpy as np
import pandas as pd

# X e y
X = datos.drop(columns='CLIENTE_MORA')
y = datos['CLIENTE_MORA']

# Detecta columnas
cat_cols = X.select_dtypes(include=['object','category']).columns.tolist()
num_cols = X.select_dtypes(include=[np.number]).columns.tolist()

# OneHotEncoder compatible (1.2-1.5+)
try:
    ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)  # sklearn ≥1.2
except TypeError:
    ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)         # sklearn <1.2

preprocess = ColumnTransformer([
    ('num', RobustScaler(), num_cols),
    ('cat', ohe, cat_cols)
], remainder='drop')

X_pre = preprocess.fit_transform(X)
print(f"Shape original: {X.shape} → transformado: {X_pre.shape}")



## Partición Train Test

In [ ]:
from sklearn.model_selection import train_test_split

seed = 2025
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.30, random_state=seed, stratify=y, shuffle=True
)

# asegurar que preprocessing_pipeline exista
try:
    preprocessing_pipeline
except NameError:
    preprocessing_pipeline = preprocess

print(f"Split OK → X_train: {X_train.shape}, X_test: {X_test.shape}, seed={seed}")
print("preprocessing_pipeline definido ✅")




## Funciones de evaluación

In [ ]:
def evaluar_modelo(best_model, X_train, y_train, X_test, y_test, nombre="Modelo", pos_label=1):
    """Imprime métricas y devuelve un dict con resultados clave."""
    # Predicciones duras
    pred_train = best_model.predict(X_train)
    pred_test  = best_model.predict(X_test)

    # Probabilidades para AUC y PR-AUC
    if hasattr(best_model, "predict_proba"):
        proba_test = best_model.predict_proba(X_test)[:, 1]
    else:
        proba_test = best_model.decision_function(X_test)

    # Métricas
    train_acc    = accuracy_score(y_train, pred_train)
    test_acc     = accuracy_score(y_test,  pred_test)
    train_recall = recall_score(y_train, pred_train, pos_label=pos_label, zero_division=0)
    test_recall  = recall_score(y_test,  pred_test,  pos_label=pos_label, zero_division=0)
    train_prec   = precision_score(y_train, pred_train, pos_label=pos_label, zero_division=0)
    test_prec    = precision_score(y_test,  pred_test,  pos_label=pos_label, zero_division=0)
    roc_test     = roc_auc_score(y_test, proba_test)
    pr_test      = average_precision_score(y_test, proba_test)

    # Reporte
    print(f"=== {nombre} ===")
    print('Train Accuracy  : ', train_acc)
    print('Test  Accuracy  : ', test_acc)
    print('Train Recall    : ', train_recall)
    print('Test  Recall    : ', test_recall)
    print('Train Precision : ', train_prec)
    print('Test  Precision : ', test_prec)
    print('ROC AUC (test)  : ', roc_test)

    print('\nConfusion Matrix:')
    print(confusion_matrix(y_test, pred_test))

    print('\nClassification Report:')
    print(classification_report(y_test, pred_test, zero_division=0))

    return {
        "train_acc":    train_acc,
        "test_acc":     test_acc,
        "train_recall": train_recall,
        "test_recall":  test_recall,
        "train_prec":   train_prec,
        "test_prec":    test_prec,
        "roc_auc":      roc_test
    }


# Definicion Hiperparámetros y CV

In [ ]:
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier

seed = globals().get('seed', 42)

param_grid = {
    "model": [DecisionTreeClassifier(random_state=seed)],
    "model__criterion": ["gini", "entropy"],
    "model__splitter": ["best", "random"],
    "model__max_leaf_nodes": [128, 256, 512, 1024],
    "model__max_depth": list(map(int, np.linspace(4, 16, 32)))
}

scoring = {'AUC': 'roc_auc', 'Accuracy': make_scorer(accuracy_score)}
kfold = StratifiedKFold(n_splits=10, random_state=seed, shuffle=True)
n_iter = 50


## Pipeline base (sin balanceo)

In [ ]:
if not any(v not in globals() for v in ["X_train","X_test","y_train","y_test","preprocessing_pipeline"]):
    full_pipeline_steps = [
        ('preprocessing_pipeline', preprocessing_pipeline),
        ('model', DecisionTreeClassifier(random_state=seed))
    ]
    full_pipeline = Pipeline(steps=full_pipeline_steps)

    grid_base = RandomizedSearchCV(
        estimator=full_pipeline,
        param_distributions=param_grid,
        cv=kfold,
        scoring=scoring,
        n_jobs=-1,
        n_iter=n_iter,
        refit="AUC",
        random_state=seed
    )
    best_model_base = grid_base.fit(X_train, y_train)
    print(">> Mejor AUC (CV) – Base:", best_model_base.best_score_)

    # === Métricas adicionales: Recall y AUC en train y test ===
    est = best_model_base.best_estimator_

    # Predicciones duras
    pred_train = est.predict(X_train)
    pred_test  = est.predict(X_test)

    # Scores probabilísticos para AUC
    if hasattr(est, "predict_proba"):
        score_train = est.predict_proba(X_train)[:, 1]
        score_test  = est.predict_proba(X_test)[:, 1]
    else:
        score_train = est.decision_function(X_train)
        score_test  = est.decision_function(X_test)

else:
    print("⛔ Define X_train/X_test/y_train/y_test/preprocessing_pipeline antes de ejecutar esta celda.")


## Undersampling dentro del CV

In [ ]:
if not any(v not in globals() for v in ["X_train","X_test","y_train","y_test","preprocessing_pipeline"]):
    undersampler = RandomUnderSampler(random_state=seed)

    pipe_under = ImbPipeline(steps=[
        ('preprocessing_pipeline', preprocessing_pipeline),
        ('sampler', undersampler),
        ('model', DecisionTreeClassifier(random_state=seed))
    ])

    grid_under = RandomizedSearchCV(
        estimator=pipe_under,
        param_distributions=param_grid,
        cv=kfold,
        scoring=scoring,
        n_jobs=-1,
        n_iter=n_iter,
        refit="AUC",
        random_state=seed
    )

    best_model_under = grid_under.fit(X_train, y_train)
    print(">> Mejor AUC (CV) – Undersampling:", best_model_under.best_score_)
else:
    print("⛔ Define X_train/X_test/y_train/y_test/preprocessing_pipeline antes de ejecutar esta celda.")


## Oversampling dentro del CV

In [ ]:
if not any(v not in globals() for v in ["X_train","X_test","y_train","y_test","preprocessing_pipeline"]):
    oversampler = RandomOverSampler(random_state=seed)

    pipe_over = ImbPipeline(steps=[
        ('preprocessing_pipeline', preprocessing_pipeline),
        ('sampler', oversampler),
        ('model', DecisionTreeClassifier(random_state=seed))
    ])

    grid_over = RandomizedSearchCV(
        estimator=pipe_over,
        param_distributions=param_grid,
        cv=kfold,
        scoring=scoring,
        n_jobs=-1,
        n_iter=n_iter,
        refit="AUC",
        random_state=seed
    )

    best_model_over = grid_over.fit(X_train, y_train)
    print(">> Mejor AUC (CV) – Oversampling:", best_model_over.best_score_)
else:
    print("⛔ Define X_train/X_test/y_train/y_test/preprocessing_pipeline antes de ejecutar esta celda.")


## SMOTE dentro del CV

In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.metrics import recall_score, roc_auc_score  # por si faltan

if not any(v not in globals() for v in ["X_train","X_test","y_train","y_test","preprocessing_pipeline"]):
    smote = SMOTE(random_state=seed, k_neighbors=5)  # puedes tunear k_neighbors

    pipe_smote = ImbPipeline(steps=[
        ('preprocessing_pipeline', preprocessing_pipeline),
        ('sampler', smote),
        ('model', DecisionTreeClassifier(random_state=seed))
    ])

    grid_smote = RandomizedSearchCV(
        estimator=pipe_smote,
        param_distributions=param_grid,
        cv=kfold,
        scoring=scoring,
        n_jobs=-1,
        n_iter=n_iter,
        refit="AUC",
        random_state=seed
    )

    best_model_smote = grid_smote.fit(X_train, y_train)
    print(">> Mejor AUC (CV) – SMOTE:", best_model_smote.best_score_)

    # Métricas rápidas (train/test) para Recall y AUC
    est = best_model_smote.best_estimator_
    pred_train = est.predict(X_train)
    pred_test  = est.predict(X_test)

    if hasattr(est, "predict_proba"):
        score_train = est.predict_proba(X_train)[:, 1]
        score_test  = est.predict_proba(X_test)[:, 1]
    else:
        score_train = est.decision_function(X_train)
        score_test  = est.decision_function(X_test)

    print(f"Train Recall: {recall_score(y_train, pred_train, zero_division=0):.4f} | "
          f"Train AUC: {roc_auc_score(y_train, score_train):.4f}")
    print(f"Test  Recall: {recall_score(y_test,  pred_test,  zero_division=0):.4f} | "
          f"Test  AUC:  {roc_auc_score(y_test,  score_test):.4f}")
else:
    print("⛔ Define X_train/X_test/y_train/y_test/preprocessing_pipeline antes de ejecutar esta celda.")

## Validaciones finales

In [ ]:
# ===== 10. Validaciones finales (incluye SMOTE si existe) =====
from sklearn.metrics import (
    accuracy_score, recall_score, roc_auc_score, average_precision_score,
    confusion_matrix, classification_report
)
models_to_eval = []
if 'best_model_base'  in globals(): models_to_eval.append(("Árbol – Base (sin balanceo)", best_model_base.best_estimator_))
if 'best_model_under' in globals(): models_to_eval.append(("Árbol – Undersampling",        best_model_under.best_estimator_))
if 'best_model_over'  in globals(): models_to_eval.append(("Árbol – Oversampling",         best_model_over.best_estimator_))
if 'best_model_smote' in globals(): models_to_eval.append(("Árbol – SMOTE",                best_model_smote.best_estimator_))

if models_to_eval:
    metrics_map = {}
    for nombre, est in models_to_eval:
        metrics_map[nombre] = evaluar_modelo(est, X_train, y_train, X_test, y_test, nombre=nombre)

    # (opcional) dejar variables sueltas como antes:
    if 'best_model_base'  in globals(): metrics_base  = metrics_map["Árbol – Base (sin balanceo)"]
    if 'best_model_under' in globals(): metrics_under = metrics_map["Árbol – Undersampling"]
    if 'best_model_over'  in globals(): metrics_over  = metrics_map["Árbol – Oversampling"]
    if 'best_model_smote' in globals(): metrics_smote = metrics_map["Árbol – SMOTE"]
else:
    print("⛔ No hay modelos para evaluar. Ejecuta primero Base/Under/Over/SMOTE.")

In [ ]:
metrics_base

## Escoger el mejor modelo y extraer el árbol final

In [ ]:
candidatos = []

if 'best_model_base'  in globals() and 'metrics_base'  in globals():
    candidatos.append(("base",  best_model_base,  metrics_base["test_recall"]))
if 'best_model_under' in globals() and 'metrics_under' in globals():
    candidatos.append(("under", best_model_under, metrics_under["test_recall"]))
if 'best_model_over'  in globals() and 'metrics_over'  in globals():
    candidatos.append(("over",  best_model_over,  metrics_over["test_recall"]))
if 'best_model_smote' in globals() and 'metrics_smote' in globals():
    candidatos.append(("smote", best_model_smote, metrics_smote["test_recall"]))

if candidatos:
    mejor_nombre, best_model, best_rec = max(candidatos, key=lambda x: x[2])
    print(f"\n>>> Mejor enfoque según Recall (test): {mejor_nombre.upper()} con Recall={best_rec:.4f}")
    dt_model = best_model.best_estimator_['model']
    print("\nHiperparámetros del árbol ganador:\n", dt_model.get_params())
else:
    print("⛔ No hay candidatos para comparar. Corre las celdas de entrenamiento/validación primero.")


## Curvas ROC y Precision-Recall del modelo ganador

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import (
    confusion_matrix, ConfusionMatrixDisplay,
    roc_curve, roc_auc_score,
    precision_recall_curve, average_precision_score
)

# === Modelo ganador: best_model_under ===
best_est = best_model_under.best_estimator_

# Probabilidades y predicciones en test
y_proba = best_est.predict_proba(X_test)[:,1]
y_pred  = (y_proba >= 0.5).astype(int)

# === Matriz de confusión ===
cm = confusion_matrix(y_test, y_pred)
ConfusionMatrixDisplay(cm).plot(cmap="Blues")
plt.title("Matriz de confusión – Modelo UNDER (umbral 0.5)")
plt.show()

# === Curva ROC ===
fpr, tpr, _ = roc_curve(y_test, y_proba)
roc_auc = roc_auc_score(y_test, y_proba)

plt.figure()
plt.plot(fpr, tpr, label=f"ROC (AUC={roc_auc:.3f})", lw=2)
plt.plot([0,1],[0,1],"--", color="gray")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate (Recall)")
plt.title("Curva ROC – Modelo UNDER")
plt.legend()
plt.grid(True)
plt.show()

# === Curva Precision–Recall ===
prec, rec, _ = precision_recall_curve(y_test, y_proba)
ap = average_precision_score(y_test, y_proba)

plt.figure()
plt.plot(rec, prec, label=f"PR (AP={ap:.3f})", lw=2, color="darkorange")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Curva Precision–Recall – Modelo UNDER")
plt.legend()
plt.grid(True)
plt.show()


# **Modelo XGBOOST CON REGULARIZACION**

## Balanceo UNDER

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
import pandas as pd

under = RandomUnderSampler(sampling_strategy=1.0, random_state=seed)
X_train_under, y_train_under = under.fit_resample(X_train, y_train)

# Resúmenes
def resumen(y):
    c = y.value_counts().sort_index()
    p = y.value_counts(normalize=True).sort_index().mul(100).round(2)
    return pd.DataFrame({"count": c, "pct%": p})

print("UNDER aplicado")
print(f"Tamaño original   : {X_train.shape[0]} filas")
print(f"Tamaño balanceado : {X_train_under.shape[0]} filas\n")

print("Distribución original (conteo y %):")
display(resumen(y_train))

print("Distribución balanceada (conteo y %):")
display(resumen(y_train_under))


## XGBoost SIN regularización

In [ ]:
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline

xgb_base = Pipeline(steps=[
    ('prep', preprocessing_pipeline),
    ('clf', XGBClassifier(
        n_estimators=300, max_depth=4, learning_rate=0.1,
        subsample=0.8, colsample_bytree=0.8,
        reg_lambda=0, reg_alpha=0,                # sin regularización
        eval_metric="auc", random_state=seed
    ))
])

xgb_base.fit(X_train_under, y_train_under)
metrics_xgb_base = evaluar_modelo(xgb_base, X_train_under, y_train_under, X_test, y_test, "XGB – Base (sin reg)")

## XGBoost con Ridge (L2: reg_lambda)

In [ ]:
xgb_ridge = Pipeline(steps=[
    ('prep', preprocessing_pipeline),
    ('clf', XGBClassifier(
        n_estimators=300, max_depth=4, learning_rate=0.1,
        subsample=0.8, colsample_bytree=0.8,
        reg_lambda=10, reg_alpha=0,               # Ridge
        eval_metric="auc", random_state=seed
    ))
])

xgb_ridge.fit(X_train_under, y_train_under)
metrics_xgb_ridge = evaluar_modelo(xgb_ridge, X_train_under, y_train_under, X_test, y_test, "XGB – Ridge (L2)")


## XGBoost con Lasso (L1: reg_alpha)

In [ ]:
xgb_lasso = Pipeline(steps=[
    ('prep', preprocessing_pipeline),
    ('clf', XGBClassifier(
        n_estimators=300, max_depth=4, learning_rate=0.1,
        subsample=0.8, colsample_bytree=0.8,
        reg_lambda=0, reg_alpha=10,               # Lasso
        eval_metric="auc", random_state=seed
    ))
])

xgb_lasso.fit(X_train_under, y_train_under)
metrics_xgb_lasso = evaluar_modelo(xgb_lasso, X_train_under, y_train_under, X_test, y_test, "XGB – Lasso (L1)")



## XGBoost con Elastic Net (L1 + L2)

In [ ]:
xgb_elastic = Pipeline(steps=[
    ('prep', preprocessing_pipeline),
    ('clf', XGBClassifier(
        n_estimators=300, max_depth=4, learning_rate=0.1,
        subsample=0.8, colsample_bytree=0.8,
        reg_lambda=5, reg_alpha=5,                # Elastic Net
        eval_metric="auc", random_state=seed
    ))
])

xgb_elastic.fit(X_train_under, y_train_under)
metrics_xgb_elastic = evaluar_modelo(xgb_elastic, X_train_under, y_train_under, X_test, y_test, "XGB – Elastic Net")


## Comparación de métricas (Accuracy, Recall, Precision, AUC)

In [ ]:
import pandas as pd

df_xgb = pd.DataFrame([
    {"Modelo": "XGB Base",        **metrics_xgb_base},
    {"Modelo": "XGB Ridge (L2)",  **metrics_xgb_ridge},
    {"Modelo": "XGB Lasso (L1)",  **metrics_xgb_lasso},
    {"Modelo": "XGB Elastic",     **metrics_xgb_elastic},
])
display(df_xgb)


## Curvas ROC comparativas (con AUC)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, roc_auc_score

plt.figure(figsize=(6,5))
for nombre, modelo in [
    ("XGB Base", xgb_base),
    ("XGB Ridge (L2)", xgb_ridge),
    ("XGB Lasso (L1)", xgb_lasso),
    ("XGB Elastic", xgb_elastic),
]:
    y_proba = modelo.predict_proba(X_test)[:,1]
    fpr, tpr, _ = roc_curve(y_test, y_proba)
    auc = roc_auc_score(y_test, y_proba)
    plt.plot(fpr, tpr, lw=2, label=f"{nombre} (AUC={auc:.3f})")

plt.plot([0,1],[0,1],'--',color='gray')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate (Recall)")
plt.title("Curvas ROC – XGBoost (Base vs Regularizaciones)")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve, average_precision_score

plt.figure(figsize=(6,5))

for nombre, modelo in [
    ("XGB Base", xgb_base),
    ("XGB Ridge (L2)", xgb_ridge),
    ("XGB Lasso (L1)", xgb_lasso),
    ("XGB Elastic", xgb_elastic),
]:
    y_proba = modelo.predict_proba(X_test)[:, 1]
    prec, rec, _ = precision_recall_curve(y_test, y_proba)
    ap = average_precision_score(y_test, y_proba)
    plt.plot(rec, prec, lw=2, label=f"{nombre} (AP={ap:.3f})")

# línea base: proporción positiva en test
pos_rate = (y_test == 1).mean()
plt.hlines(pos_rate, 0, 1, linestyles='--', colors='gray', label=f"Base rate = {pos_rate:.2f}")

plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Curvas Precision–Recall – XGBoost (Base vs Regularizaciones)")
plt.legend()
plt.grid(True)
plt.xlim(0, 1); plt.ylim(0, 1)
plt.show()


In [ ]:
# Elegir mejores modelos por AUC y por Recall (test)
candidatos = [
    ("XGB Base",    xgb_base,    metrics_xgb_base),
    ("XGB Ridge",   xgb_ridge,   metrics_xgb_ridge),
    ("XGB Lasso",   xgb_lasso,   metrics_xgb_lasso),
    ("XGB Elastic", xgb_elastic, metrics_xgb_elastic),
]

# Mejor por AUC
mejor_auc_nombre, mejor_auc_modelo, mejor_auc_score = max(
    ((n, m, met["roc_auc"]) for n, m, met in candidatos),
    key=lambda t: t[2]
)

# Mejor por Recall (test)
mejor_rec_nombre, mejor_rec_modelo, mejor_rec_score = max(
    ((n, m, met["test_recall"]) for n, m, met in candidatos),
    key=lambda t: t[2]
)

print(f">>> Mejor por AUC     : {mejor_auc_nombre}  | AUC={mejor_auc_score:.4f}")
print(f">>> Mejor por Recall  : {mejor_rec_nombre} | Recall={mejor_rec_score:.4f}")

# (Opcional) hiperparámetros internos del clasificador ganador en cada criterio
print("\nHiperparámetros – ganador por AUC:")
print(mejor_auc_modelo.named_steps['clf'].get_params())

print("\nHiperparámetros – ganador por Recall:")
print(mejor_rec_modelo.named_steps['clf'].get_params())

# (Opcional) dejar variables de salida para usar después
best_by_auc     = {"nombre": mejor_auc_nombre, "modelo": mejor_auc_modelo, "score": mejor_auc_score}
best_by_recall  = {"nombre": mejor_rec_nombre, "modelo": mejor_rec_modelo, "score": mejor_rec_score}



# Código completo XGBOOST con Undersampling y Regularización L1, cross_validation y Optimización del umbral.

In [ ]:
# ===============================================
# 1️⃣ BALANCEO DE CLASES (UNDERSAMPLING)
# ===============================================
from imblearn.under_sampling import RandomUnderSampler

# Se iguala la cantidad de clases para evitar sesgo hacia la clase mayoritaria
under = RandomUnderSampler(sampling_strategy=1.0, random_state=seed)
X_train_under, y_train_under = under.fit_resample(X_train, y_train)

# Mostrar tamaños y proporciones
print("UNDER aplicado correctamente ✅")
print(f"Tamaño original: {X_train.shape[0]} filas")
print(f"Tamaño balanceado: {X_train_under.shape[0]} filas\n")

print("Distribución original:")
print(y_train.value_counts())
print("\nDistribución balanceada:")
print(y_train_under.value_counts())

In [ ]:
# ===============================================
# 2️⃣ MODELO XGBOOST CON REGULARIZACIÓN LASSO (L1)
# ===============================================
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline

# Regularización L1 se controla con reg_alpha > 0
xgb_lasso = Pipeline(steps=[
    ('prep', preprocessing_pipeline),
    ('clf', XGBClassifier(
        n_estimators=300, max_depth=4, learning_rate=0.1,
        subsample=0.8, colsample_bytree=0.8,
        reg_lambda=0, reg_alpha=10,               # Lasso (L1)
        eval_metric="auc", random_state=seed
    ))
])

print("Modelo XGBoost con regularización L1 configurado ✅")

In [ ]:
# ===============================================
# 3️⃣ VALIDACIÓN CRUZADA (STRATIFIED K-FOLD)
# ===============================================
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_recall_curve, roc_auc_score, average_precision_score
import numpy as np

# Se define validación cruzada estratificada (mantiene proporciones de clases)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

# Vector para guardar probabilidades OOF (out-of-fold)
oof_proba = np.zeros(len(y_train_under), dtype=float)

for tr_idx, va_idx in cv.split(X_train_under, y_train_under):
    xtr, xva = X_train_under.iloc[tr_idx], X_train_under.iloc[va_idx]
    ytr, yva = y_train_under.iloc[tr_idx], y_train_under.iloc[va_idx]
    
    xgb_lasso.fit(xtr, ytr)
    oof_proba[va_idx] = xgb_lasso.predict_proba(xva)[:, 1]

# Calcular AUC promedio de la validación cruzada
auc_cv = roc_auc_score(y_train_under, oof_proba)
ap_cv  = average_precision_score(y_train_under, oof_proba)

print(f"AUC promedio CV: {auc_cv:.3f}")
print(f"Average Precision (PR-AUC) CV: {ap_cv:.3f}")

In [ ]:
# ===============================================
# 4️⃣ OPTIMIZACIÓN DEL UMBRAL (THRESHOLD)
# ===============================================
from sklearn.metrics import f1_score

# Calculamos curva Precision-Recall
prec, rec, thr = precision_recall_curve(y_train_under, oof_proba)
f1 = 2 * prec * rec / (prec + rec + 1e-12)
thr_candidates = np.r_[thr, 1.0]  # Alinear longitudes

# Seleccionamos el umbral que maximiza el F1
idx_opt = np.nanargmax(f1)
thr_opt = float(thr_candidates[idx_opt])

print(f"Umbral óptimo seleccionado: {thr_opt:.3f}")
print(f"F1 óptimo: {f1[idx_opt]:.3f} | Precisión: {prec[idx_opt]:.3f} | Recall: {rec[idx_opt]:.3f}")

In [ ]:
# ===============================================
# 5️⃣ REENTRENAMIENTO Y EVALUACIÓN FINAL EN TEST
# ===============================================

# Entrenamos el modelo con todos los datos balanceados
xgb_lasso.fit(X_train_under, y_train_under)

# Calculamos probabilidades y predicciones en Test con el umbral óptimo
proba_test = xgb_lasso.predict_proba(X_test)[:, 1]
pred_test  = (proba_test >= thr_opt).astype(int)

# Evaluamos con la función definida previamente
metrics_xgb_lasso_opt = evaluar_modelo(xgb_lasso, X_train_under, y_train_under, X_test, y_test, nombre="XGBoost Lasso (L1) – Threshold Óptimo")